In [1]:
import pandas as pd
import os
import numpy as np
import arcpy

In [18]:
ica = pd.read_csv(r'data/ic.txt')

In [3]:
out = {}

for i, row in ica.iterrows():
    out[row[3]] = {'NAZOB': row[2], 'OBYV11': row[4]}

In [4]:
for year in os.listdir(r'data/finm/'):
    print(year)
    temp = pd.read_csv(r'data/finm/' + year, sep=';', low_memory=False, dtype='str')
    if(int(year.split('_')[0]) <= 2012):
        adder = '00'
    else:
        adder = ''
    
    for obec in out:
        try:
            saldo = temp[(temp['ZC_ICO:ZC_ICO'] == adder + obec) & (temp['ZC_POLVYK:ZC_POLVYK'] == '4440')][['ZU_ROZKZ:ZU_ROZKZ']].iloc[0]['ZU_ROZKZ:ZU_ROZKZ']
            if(saldo.endswith('-')):
                sal = float(saldo.rstrip('-')) * -1.0
            else:
                sal = float(saldo)
            out[obec]['saldo_' + year.split('_')[0]] = sal
        except:
            out[obec]['saldo_' + year.split('_')[0]] = np.nan
            print(obec, year)

2010_FINM203.csv
  2010_FINM203.csv
01265750 2010_FINM203.csv
72086718 2010_FINM203.csv
01265741 2010_FINM203.csv
00017361 2010_FINM203.csv
00241989 2010_FINM203.csv
2011_FINM203.csv
  2011_FINM203.csv
01265750 2011_FINM203.csv
01265741 2011_FINM203.csv
00017361 2011_FINM203.csv
00241989 2011_FINM203.csv
2012_FINM203.csv
  2012_FINM203.csv
01265750 2012_FINM203.csv
01265741 2012_FINM203.csv
00017361 2012_FINM203.csv
00241989 2012_FINM203.csv
2013_FINM203.csv
  2013_FINM203.csv
00017361 2013_FINM203.csv
00241989 2013_FINM203.csv
2014_FINM203.csv
  2014_FINM203.csv
00017361 2014_FINM203.csv
00241989 2014_FINM203.csv


In [5]:
data = pd.DataFrame.from_dict(out, orient='index').reset_index().rename(columns={'index': 'ico'})

In [29]:
#nahrat do GDB
def toGDB(dframe, fclass):
    arr_values = {}
    arr_dtypes = []

    cols = dframe.columns

    for col in cols:

        arr = dframe[col].values

        if arr.dtype == np.object:
            arr = arr.astype(np.unicode)

        if arr.dtype == np.int64:
            max_val = arr.max()
            min_val = arr.min()
            if min_val < -2147483647 or max_val > 2147483647:
                arr = arr.astype(np.float64)
            else:
                arr = arr.astype(np.int32)

        if arr.dtype == np.bool:
            arr = arr.astype(np.int32)

        arr_values[col] = arr
        arr_dtypes.append((col, arr.dtype))

    outArr = np.empty(len(dframe), dtype=arr_dtypes)

    for col in arr_values:
        outArr[col] = arr_values[col]

    arcpy.da.NumPyArrayToTable(outArr, fclass)

In [ ]:
toGDB(data, r'data/ob_rozpocty.gdb/obce_saldo')

In [4]:
#alias rename
fclass = "data/ob_rozpocty.gdb/obce_saldo"
fieldnames = [f.name for f in arcpy.ListFields(fclass)]

for column in fieldnames:
    if (column.startswith('saldo')):
        arcpy.AlterField_management(in_table=r'data/ob_rozpocty.gdb/obce_saldo', field=column, new_field_alias=column)

In [27]:
#pocty obyvatel
outo = {}

for i, row in ica.iterrows():
    outo[row[3]] = {'ICOB': row[1], 'NAZOB': row[2]}

obyvOut = pd.DataFrame.from_dict(outo, orient='index')

for file in os.listdir(r'data/obyv/'):
    temp = pd.read_excel(r'data/obyv/' + file, skiprows=6, parse_cols=[1, 3]).dropna(subset=['Unnamed: 1']).rename(columns={'Unnamed: 0': 'ICOB','Unnamed: 1': file.split('.')[0],})
    obyvOut = obyvOut.merge(temp, on='ICOB', how='left')

In [30]:
toGDB(obyvOut, r'data/ob_rozpocty.gdb/obce_obyvatele')

In [6]:
arcpy.CalculateField_management(in_table="data/ob_rozpocty.gdb/obce_saldo_poly", field="saldo_obyv_2014", expression="!saldo_2014! / !obyv_2014!", expression_type="PYTHON_9.3", code_block="")

<Result 'data/ob_rozpocty.gdb/obce_saldo_poly'>

In [ ]:
#data.to_excel(r'data/sumarized.xlsx')